In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances

seed = 42

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.spatial import distance
from scipy import sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
sns.set_color_codes()

import warnings
warnings.filterwarnings('ignore')

data_dir = '/projects/Colgate_Project/clustering_data/'

In [2]:
df = pd.read_csv(os.path.join(data_dir,'rc_data_v9.csv'))
df.head()

,retailer_join,year,quarter,sub_re,act_dist,amin,amax,sum,mean,median,...,Colgate Super Shine Man TB,Colgate Superfresh TP,Colgate Total Man TB,Colgate Total TP,Colgate Vitamin C TP,Colgate Zigzag Man TB,Palmolive Naturals BW,Palmolive Naturals Shampoo,Promo Matl Other SB,Softlan DILUTE Liq FC
0,1A0005,2016,Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1A0005,2016,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1A0005,2016,Q3,SMN,10100577.0,321000.0,9288000.0,1.169673e+08,2.436819e+06,1579800.0,...,0.0,132.0,0.0,0.0,168.0,0.0,0.0,24.0,0.0,0.0
3,1A0005,2016,Q4,SMN,10100577.0,510000.0,13003200.0,2.225726e+08,2.119739e+06,1440000.0,...,0.0,552.0,0.0,0.0,264.0,0.0,192.0,72.0,0.0,36.0
4,1A0005,2017,Q1,SMN,10100577.0,306000.0,6796800.0,2.015864e+08,1.919870e+06,1435200.0,...,0.0,336.0,0.0,120.0,204.0,0.0,36.0,108.0,0.0,0.0


In [3]:
df = df.fillna(0)

In [4]:
df.head()

,retailer_join,year,quarter,sub_re,act_dist,amin,amax,sum,mean,median,...,Colgate Super Shine Man TB,Colgate Superfresh TP,Colgate Total Man TB,Colgate Total TP,Colgate Vitamin C TP,Colgate Zigzag Man TB,Palmolive Naturals BW,Palmolive Naturals Shampoo,Promo Matl Other SB,Softlan DILUTE Liq FC
0,1A0005,2016,Q1,0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1A0005,2016,Q2,0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1A0005,2016,Q3,SMN,10100577.0,321000.0,9288000.0,1.169673e+08,2.436819e+06,1579800.0,...,0.0,132.0,0.0,0.0,168.0,0.0,0.0,24.0,0.0,0.0
3,1A0005,2016,Q4,SMN,10100577.0,510000.0,13003200.0,2.225726e+08,2.119739e+06,1440000.0,...,0.0,552.0,0.0,0.0,264.0,0.0,192.0,72.0,0.0,36.0
4,1A0005,2017,Q1,SMN,10100577.0,306000.0,6796800.0,2.015864e+08,1.919870e+06,1435200.0,...,0.0,336.0,0.0,120.0,204.0,0.0,36.0,108.0,0.0,0.0


In [5]:
ret_groups = df.groupby(['sub_re', 'act_dist', 'year', 'quarter'])
ret_groups = sorted(ret_groups, key=lambda x: len(x[1]), reverse=True) #order the retailer groups

In [6]:
#clustering the retailers within groups according to previous two quarters.

def get_prev_2_quarters(df, year, quarter):
    if quarter == 'Q1':
        quarters = [(year-1, 'Q4'), (year-1, 'Q3')]
    if quarter == 'Q2':
        quarters = [(year, 'Q1'), (year-1, 'Q4')]
    if quarter == 'Q3':
        quarters = [(year, 'Q2'), (year, 'Q1')]
    if quarter == 'Q4':
        quarters = [(year, 'Q3'), (year, 'Q2')]
    df_prev = df[(df['year'] == quarters[0][0]) & (df['quarter'] == quarters[0][1])]
    df_prev_1 = df[(df['year'] == quarters[1][0]) & (df['quarter'] == quarters[1][1])]
#     unweighted_cols = ['retailer_join', 'sub_re', 'act_dist', 'year', 'quarter']
#     df_prev.iloc[:,5:] = df_prev.iloc[:,5:].mul(w1)
#     df_prev_1.iloc[:,5:] = df_prev_1.iloc[:,5:].mul(w2)
    return pd.merge(df_prev, df_prev_1, 
                    on=['sub_re', 'act_dist', 'retailer_join'], 
                    suffixes=('_prev', '_prev-1')).drop_duplicates(keep='first')


In [7]:
def prep_data_for_group(X, w1, w2):
    #drop column if all values are 0s
    X = X.loc[:, (X != 0).any(axis=0)]
    prev_cols = [col for col in X.columns if col.endswith('prev')]
    prev_1_cols = [col for col in X.columns if col.endswith('prev-1')]
    X[prev_cols] = MinMaxScaler(feature_range=(0, w1)).fit_transform(X[prev_cols])
    X[prev_1_cols] = MinMaxScaler(feature_range=(0, w2)).fit_transform(X[prev_1_cols])
    return X

In [8]:
#encode categorical features. Write it just in case for future use. 
def encode_cat_features(X, cat_cols):
    for col in cat_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col]) 
        X[col] = X[col].astype('category')
        X[col] = X[col].cat.codes
    return X

In [9]:
#Use SVD for reducing dimensionality.
def reduce_dims(X_scaled):
    dims = range(2, np.shape(X_scaled)[1] - 1)
    for dim in dims:
        svd = TruncatedSVD(n_components=dim)
        X_red = svd.fit_transform(X_scaled)
        exp_variance = svd.explained_variance_ratio_.sum() #expected variance
        if exp_variance >= 0.8:
            break
    print('Dim:', dim)
    return X_red

In [10]:
#choose k with the maximum silhouette score in kmeans_train
def choose_k(k_mid, k_max, ss):
    k_range = pd.Series(range(2, k_max))
    k_min = max(k_mid-6, 0)
    ss = pd.Series(ss)
    padding = pd.Series([0])
    v = ss[(k_range < k_max) & (k_range > k_min)]
    v = pd.concat([padding, v, padding], ignore_index=True)
    x = k_range[(k_range < k_max) & (k_range > k_min)]
    x = pd.concat([padding, x, padding], ignore_index=True)
    # maxima index
    idx = np.all([(np.diff(v)[:-1] > 0), (np.diff(v[::-1])[::-1][1:] > 0)], axis = 0)
    # maxima
    maxima = max(v[1:-1][idx])
    return (x[v == maxima].iloc[0], v[v == maxima].iloc[0])

In [11]:
def kmeans_train(X_red):
    k_mid = int(len(X_red)/40)
    k_max = k_mid + 6
    k_range = range(2, k_max)
    labels_for_k = {}
    ss_for_k = {}
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=seed).fit(X_red)
        labels = kmeans.labels_
        labels_for_k[k] = labels
        ss_for_k[k] = silhouette_score(X_red, labels)
#     print(list(ss_for_k.values()))
#     plt.plot(list(ss_for_k.keys()), list(ss_for_k.values()))
    k_opt, ss_opt = choose_k(k_mid, k_max, list(ss_for_k.values()))
    
    #return the distance matrix (Euclidean distance)
    #For each cluster want to return the centroid to the other clusters; and return the closest cluster index
    kmeans_opt = KMeans(n_clusters=k_opt, random_state=seed).fit(X_red)
    
    #pairwise Euclidean distances of the centroids between clusters
    dists = euclidean_distances(kmeans_opt.cluster_centers_) 
    
    #minimum distance 
    #tri_dists = dists[np.triu_indices(k_opt, 1)]
    #min_dist = tri_dists.min()
    
    print('Total size:', len(X_red))
    print('K:', k_opt)
    print('SS:', ss_opt)
    #print('dist:', dists)
    
    return labels_for_k[k_opt], dists

Try this:
sub_re: SMN
'act_dist': 10100577 

In [22]:
"""
df['act_dist']=df['act_dist'].astype(int)
df_1=df[df['sub_re']=='SMN']
df_2 = df_1[df_1['act_dist']==10100577]
df_cluster=df_2
df_cluster.shape
"""

In [26]:
def get_clusters_for_group(df):
    X = df.drop(['retailer_join', 'sub_re', 'act_dist', 'year_prev', 'quarter_prev', 'year_prev-1', 'quarter_prev-1'], axis=1)
#     cat_cols = ['route_code', 'street_num']
#     X = encode_cat_features(X, cat_cols)
    X_scaled = prep_data_for_group(X, 0.55, 0.45)
    X_red = reduce_dims(X_scaled)
    labels,dists = kmeans_train(X_red)
    dist_matrix = dists
    clusters = df
    clusters['class'] = labels
    return clusters, dist_matrix


In [27]:
"""
all_clusters = pd.DataFrame()
dist_matrix = np.matrix
g = 1
for n1, _ in df.groupby(['year', 'quarter']):
    if (n1[0] == 2019) & (n1[1] == 'Q1'):
        q_df = get_prev_2_quarters(df_cluster, n1[0], n1[1])
        #for n2, ret_group in q_df.groupby(['sub_re', 'act_dist']):
        #    if len(ret_group) > 40: #cluster when there are at least 40 retailers within the group
        #        print(n2)
        clusters,dist_matrix = get_clusters_for_group(q_df)
        clusters['year'] = n1[0]
        clusters['quarter'] = n1[1]
        all_clusters = pd.concat([all_clusters, clusters], axis=0)
        g += 1
        print('='*50)
        break
"""

"\nall_clusters = pd.DataFrame()\ndist_matrix = np.matrix\ng = 1\nfor n1, _ in df.groupby(['year', 'quarter']):\n    if (n1[0] == 2019) & (n1[1] == 'Q1'):\n        q_df = get_prev_2_quarters(df_cluster, n1[0], n1[1])\n        #for n2, ret_group in q_df.groupby(['sub_re', 'act_dist']):\n        #    if len(ret_group) > 40: #cluster when there are at least 40 retailers within the group\n        #        print(n2)\n        clusters,dist_matrix = get_clusters_for_group(q_df)\n        clusters['year'] = n1[0]\n        clusters['quarter'] = n1[1]\n        all_clusters = pd.concat([all_clusters, clusters], axis=0)\n        g += 1\n        print('='*50)\n        break\n"

In [28]:
#This is for a single group:

#Find closest class of sparse cluster:
#dist_matrix is a symmetric matrix
def closest_class(all_clusters, dist_matrix):
    #find sparse/small clusters:
    grouped = pd.DataFrame(all_clusters.groupby('class')['retailer_join'].nunique())
    grouped['class'] = grouped.index
    
    # small class for retailer <=5
    small_class = grouped[grouped['retailer_join']<=5]['class']
    
    #Now change criterion: small cluster for transaction numbers <= 50
    
    small_class = np.array(small_class)
    
    closest_cluster = pd.DataFrame()
    closest_cluster['class'] = small_class
    closest_cluster['closest_class'] = 0
    
    for class_idx in small_class:
        distance = dist_matrix[class_idx]
        max_dist = max(distance)
        # mask the dist_matrix with the largest distance+1 for any small class 
        # because otherwise it may relate to another small class
        for i in range(len(distance)):
            if i in small_class:
                distance[i] = max_dist+1
        
        #exlude 0, the second minimum is the closest
        closest_cluster.loc[closest_cluster['class']==class_idx,\
                            ['closest_class']]=np.argsort(distance)[:2][1] 
        
        #Finally join closest_cluster with the clustering dataframe
        
    return pd.merge(all_clusters, closest_cluster, on='class', how='left')

In [29]:
all_clusters = pd.DataFrame()
dist_matrix = np.matrix

g = 1
for n1, _ in df.groupby(['year', 'quarter']):
    if (n1[0] == 2019) & (n1[1] == 'Q1'):
        q_df = get_prev_2_quarters(df_cluster, n1[0], n1[1])
        #for n2, ret_group in q_df.groupby(['sub_re', 'act_dist']):
        #    if len(ret_group) > 40: #cluster when there are at least 40 retailers within the group
        #        print(n2)
        clusters,dist_matrix = get_clusters_for_group(q_df)
        clusters['year'] = n1[0]
        clusters['quarter'] = n1[1]
        cluster_with_closest = closest_class(clusters,dist_matrix)
        all_clusters = pd.concat([all_clusters, cluster_with_closest], axis=0)
        g += 1
        print('='*50)
        break

Dim: 11
Total size: 716
K: 13
SS: 0.6733546766072629
dist: [[0.         0.68576727 0.66461562 0.74429193 1.31651595 1.58598848
  0.32089006 0.2089207  1.46128112 0.42577577 0.25227616 1.48072545
  0.4034287 ]
 [0.68576727 0.         0.40005765 0.51967228 1.13716252 1.09289294
  0.37246739 0.59367196 1.53258865 0.59912336 0.53541569 1.58947929
  0.70719116]
 [0.66461562 0.40005765 0.         0.62322179 1.21935008 1.11374639
  0.41498007 0.5978027  1.51635448 0.54879192 0.56743504 1.57627331
  0.71037957]
 [0.74429193 0.51967228 0.62322179 0.         1.01086418 0.95903496
  0.57401687 0.67152093 1.45634414 0.52951173 0.53016697 1.61378911
  0.74015956]
 [1.31651595 1.13716252 1.21935008 1.01086418 0.         1.31025578
  1.22004455 1.28356778 1.83198211 1.2607755  1.10836118 1.67924964
  1.07965215]
 [1.58598848 1.09289294 1.11374639 0.95903496 1.31025578 0.
  1.34280537 1.50263887 1.89560654 1.35818552 1.37951151 2.09734738
  1.51173992]
 [0.32089006 0.37246739 0.41498007 0.57401687 1.2

In [21]:
all_clusters.head()

,retailer_join,year_prev,quarter_prev,sub_re,act_dist,amin_prev,amax_prev,sum_prev,mean_prev,median_prev,...,Colgate Vitamin C TP_prev-1,Colgate Zigzag Man TB_prev-1,Palmolive Naturals BW_prev-1,Palmolive Naturals Shampoo_prev-1,Promo Matl Other SB_prev-1,Softlan DILUTE Liq FC_prev-1,class,year,quarter,closest_class
0,1A0005,2018,Q4,SMN,10100577,4800.05,166601.60,3448738.30,29730.502586,19680.140,...,0.0,0.0,60.0,48.0,0.0,0.0,9,2019,Q1,15.0
1,1A0009,2018,Q4,SMN,10100577,1199.88,36720.28,1030291.45,8585.762083,6716.425,...,0.0,0.0,195.0,43.0,0.0,0.0,4,2019,Q1,NaN
2,1A0010,2018,Q4,SMN,10100577,999.90,36720.28,684443.56,9249.237297,5638.775,...,0.0,0.0,0.0,18.0,0.0,0.0,15,2019,Q1,NaN
3,1A0011,2018,Q4,SMN,10100577,2699.99,55968.00,1123468.94,13700.840732,11539.180,...,0.0,0.0,60.0,0.0,0.0,0.0,15,2019,Q1,NaN
4,1A0014,2018,Q4,SMN,10100577,9900.00,24990.24,59880.48,19960.160000,24990.240,...,0.0,0.0,36.0,0.0,0.0,0.0,6,2019,Q1,NaN


## Multiple groups

In [ ]:
#Find closest class of sparse cluster:
#dist_matrix is a symmetric matrix
def closest_class(all_clusters, dist_matrix):
    #find sparse/small clusters:
    grouped = pd.DataFrame(all_clusters.groupby('class')['retailer_join'].nunique())
    grouped['class'] = grouped.index
    small_class = grouped[grouped['retailer_join']<=5]['class']
    small_class = np.array(small_class)
    
    closest_cluster = pd.DataFrame()
    closest_cluster['class'] = small_class
    closest_cluster['closest_class'] = 0
    
    for class_idx in small_class:
        distance = dist_matrix[class_idx]
        max_dist = max(distance)
        # mask the dist_matrix with the largest distance for any small class 
        # because otherwise it may relate to another small class
        for i in range(len(distance)):
            if i in small_class:
                distance[i] = max_dist + 1
        #exlude 0, the second minimum is the closest
        closest_cluster.loc[closest_cluster['class']==class_idx,\
                            ['closest_class']]=np.argsort(distance)[:2][1] 
    
        #Finally join closest_cluster with the clustering dataframe
    
    return pd.merge(all_clusters, closest_cluster, on='class', how='left')

In [ ]:
#Closest cluster with multiple groups
all_clusters = pd.DataFrame()
dist_matrix = np.matrix
g = 1
for n1, _ in df.groupby(['year', 'quarter']):
    if (n1[0] == 2019) & (n1[1] == 'Q1'):
        q_df = get_prev_2_quarters(df_cluster, n1[0], n1[1])
        for n2, ret_group in q_df.groupby(['sub_re', 'act_dist']):
            if len(ret_group) > 40: 
                print(n2)
        clusters,dist_matrix = get_clusters_for_group(q_df)
        clusters['year'] = n1[0]
        clusters['quarter'] = n1[1]
        cluster_with_closest = closest_class(clusters,dist_matrix)
        all_clusters = pd.concat([all_clusters, cluster_with_closest], axis=0)
        g += 1
        print('='*50)
        break

In [1]:
pwd

'/Users/i500577/Desktop/work/projects/Colgate_Project/colgate_dms/ds/notebooks'